In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupKFold 


IMPORTA OS DADOS

In [35]:
dados_xlsx = pd.read_excel('TCC DADOS3.xlsx', sheet_name=6)
dados_xlsx = dados_xlsx.rename(columns={'Episódio': 'ep', "Posição" : 'pos', 'participante' : 'queen' , 'ID' : 'id'})

dados_xlsx.head()




,id,queen,ep,pos
0,1,BeBe Zahara Benet,1,0
1,1,BeBe Zahara Benet,2,0
2,1,BeBe Zahara Benet,3,2
3,1,BeBe Zahara Benet,4,1
4,1,BeBe Zahara Benet,5,-2


In [ ]:
dim_queen= pd.read_excel('TCC DADOS3.xlsx', sheet_name=0)
dim_queen = dim_queen.rename(columns={"ID": "id" , "temp ":"temp", "franquia " : "franquia"})
dim_queen['temp'] = dim_queen['temp'].fillna(0).round(0).astype(int)

dim_queen['temp'] = dim_queen['temp'].round(0).astype(int)

# cria coluna combinando temp + franquia como string

dim_queen['tempfranquia'] = dim_queen['temp'].astype(str) +  '0'  +dim_queen['franquia'].astype(str)


dim_queen.head()

,id,nome,idade,colocacao,cidade,estado,temp,franquia,tempfranquia
0,1,BeBe Zahara Benet,28,1,Minneapolis,Minnesota,1,1,101
1,2,Nina Flowers,34,2,Denver,Colorado,1,1,101
2,3,Rebecca Glasscock,26,3,Fort Lauderdale,Flórida,1,1,101
3,4,Shannel,29,4,Las Vegas,Nevada,1,1,101
4,5,Ongina,26,5,Los Angeles,Califórnia,1,1,101


In [37]:
print(dim_queen.columns)

Index(['id', 'nome', 'idade', 'colocacao', 'cidade', 'estado   ', 'temp',
       'franquia', 'tempfranquia'],
      dtype='object')


In [ ]:
## conta o numero de cada posicao aqui

colunas = [
    'id', 'queen', 'ep', 'desq', 'elim', 'btm', 'low', 'safe',
    'high', 'win', 'top2', 'top3', 'runnerup', 'winner'
]

dados_completos = pd.DataFrame(columns=colunas)
desq, elim, btm, low, safe, high, win, top2, top3, runnerup, winner = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
aux = 1

for index, row in dados_xlsx.iterrows():
    
    if row['id'] != aux:
   
        desq, elim, btm, low, safe, high, win, top2, top3, runnerup, winner = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
        aux = row['id']
    if row['pos'] == -4:
        desq += 1
    elif row['pos'] == -3:
        elim += 1
    elif row['pos'] == -2:
        btm += 1
    elif row['pos'] == -1:
        low += 1
    elif row['pos'] == 0:
        safe += 1
    elif row['pos'] == 1:
        high += 1
    elif row['pos'] == 2:
        win += 1
    elif row['pos'] == 1.5:
        high += 1
    elif row['pos'] == 3:
        top3 += 1
    elif row['pos'] == 4:
        runnerup += 1
    elif row['pos'] == 5:
        winner += 1

    dados_completos.loc[len(dados_completos)] = [
    row['id'], row['queen'], row['ep'],
    desq, elim, btm, low, safe,
    high, win, top2, top3, runnerup, winner
]

    

In [ ]:
#da uma pontuação para cada colocação

colunas = [
    'id', 'queen', 'ep', 'bom', 'ruim', 'media'
]

dados_completos2 = pd.DataFrame(columns=colunas)
bom, ruim, media = 0,0,0
aux = 1

for index, row in dados_xlsx.iterrows():
    
    if row['id'] != aux:
       
        bom, ruim, media = 0,0,0
        aux = row['id']
        
    if row['pos'] == -4:
        bom += 0
    elif row['pos'] == -3:
     
        ruim += 6
        media = media - 3 
    elif row['pos'] == -2:
      
        ruim += 5
        media = media - 2
    elif row['pos'] == -1:
       
        ruim += 4
        media = media - 1
    elif row['pos'] == 0:
        bom += 1
        ruim+= 1
        
    elif row['pos'] == 1:
        bom += 4
       
        media = media + 1
    elif row['pos'] == 2:
        bom += 6
      
        media = media + 2
    elif row['pos'] == 1.5:
        bom += 6
    
        media = media +1.5
    

    dados_completos2.loc[len(dados_completos2)] = [
    row['id'], row['queen'], row['ep'],
    bom, ruim, media
]

    

In [40]:
#inner join com a dimensao para pegar outros dados
dados = dados_completos.merge(dim_queen[["id", "colocacao","idade", "tempfranquia"]], on="id", how="inner")
dados2 = dados_completos2.merge(dim_queen[["id", "colocacao","idade", "tempfranquia"]], on="id", how="inner")



In [41]:
dados2.head()

,id,queen,ep,bom,ruim,media,colocacao,idade,tempfranquia
0,1,BeBe Zahara Benet,1,1,1,0.0,1,28,101
1,1,BeBe Zahara Benet,2,2,2,0.0,1,28,101
2,1,BeBe Zahara Benet,3,8,2,2.0,1,28,101
3,1,BeBe Zahara Benet,4,12,2,3.0,1,28,101
4,1,BeBe Zahara Benet,5,12,7,1.0,1,28,101


In [42]:
# Remove duplicatas consecutivas ignorando 'ep', deixando só os eps que realmente a queen participou
dados3 = dados2.loc[
    ~(dados2.drop(columns=["ep"]).eq(dados2.drop(columns=["ep"]).shift())).all(axis=1)
]

dados2 = dados.loc[
    ~(dados.drop(columns=["ep"]).eq(dados.drop(columns=["ep"]).shift())).all(axis=1)
]


In [ ]:
# Salvar o DataFrame em um arquivo Excel 
dados3['vencedora'] = (dados3['colocacao'] == 1).astype(int)
dados2['vencedora'] = (dados2['colocacao'] == 1).astype(int)


dados3.to_excel("queen_por_pontuacao.xlsx", index=False)
dados3.to_excel("queen_por_posicao.xlsx", index=False)



C:\Users\bruno\AppData\Local\Temp\ipykernel_24220\353388747.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados2['vencedora'] = (dados2['colocacao'] == 1).astype(int)


PEGA O ULTIMO EPISÓDIO DA PARTICIPANTE, E O EPISÓDIO DO MEIO DE CADA TEMPORADA INDIVIDUALMENTE, POR FIM AGRUPA A LINHA FATO DE TODAS ELAS DO MODELO DE DADOS

In [44]:
result = dados2.groupby("tempfranquia")["ep"].max().reset_index()

# renomear a coluna para "maxep"
eps = result.rename(columns={"ep": "maxep"})
eps["metadeep"] = (eps["maxep"] / 2).astype(int)


result2=dados3.groupby("tempfranquia")["ep"].max().reset_index()
result2 

eps2 = result2.rename(columns={"ep": "maxep"})
eps2["metadeep"] = (eps2["maxep"] / 2).astype(int)
eps2 



,tempfranquia,maxep,metadeep
0,1001,12,6
1,101,6,3
2,1011,7,3
3,1012,7,3
4,1013,5,2
5,1014,9,4
6,1015,7,3
7,1016,10,5
8,1018,10,5
9,102,7,3


In [69]:
print(dados[dados['tempfranquia'] == '203'])


       id     queen  ep  desq  elim  btm  low  safe  high  win  top2  top3  \
9253  645  Ruby Nox   1     0     0    0    0     0     0    1     0     0   
9254  645  Ruby Nox   2     0     0    0    0     1     0    1     0     0   
9255  645  Ruby Nox   3     0     0    0    0     2     0    1     0     0   
9256  645  Ruby Nox   4     0     0    0    0     3     0    1     0     0   
9257  645  Ruby Nox   5     0     0    0    0     4     0    1     0     0   
...   ...       ...  ..   ...   ...  ...  ...   ...   ...  ...   ...   ...   
9418  654    Chanel  13     0     1    0    0     0     1    0     0     0   
9419  654    Chanel  14     0     1    0    0     0     1    0     0     0   
9420  654    Chanel  15     0     1    0    0     0     1    0     0     0   
9421  654    Chanel  16     0     1    0    0     0     1    0     0     0   
9422  654    Chanel  17     0     1    0    0     0     1    0     0     0   

      runnerup  winner  colocacao  idade tempfranquia  
9253   

In [ ]:


dados3= pd.read_excel('queen_por_pontuacao.xlsx', sheet_name=0)
queen_por_posicao= pd.read_excel('queen_por_pontuacao.xlsx', sheet_name=0)

In [70]:
agrupa_maior_ep = dados3.assign(ep=dados3["ep"].astype(int)).groupby("tempfranquia")["ep"].max().reset_index()

#agrupa_maior_ep


maxep_metadeep = agrupa_maior_ep.rename(columns={"ep": "maxep"})
maxep_metadeep["metadeep"] = (maxep_metadeep["maxep"] / 2).astype(int)
maxep_metadeep.head()


,tempfranquia,maxep,metadeep
0,1001,12,6
1,101,6,3
2,1011,7,3
3,1012,7,3
4,1013,5,2


In [72]:
dados_meio= pd.merge(
    dados3,
    maxep_metadeep,
    left_on=["tempfranquia", "ep"],
    right_on=["tempfranquia", "metadeep"],
    how="inner"
)
dados_meio=dados_meio.drop(['ep', 'metadeep', 'maxep','colocacao'], axis=1)
dados_meio.head()

,id,queen,bom,ruim,media,idade,tempfranquia,vencedora
0,1,BeBe Zahara Benet,8,2,2.0,28,101,1
1,2,Nina Flowers,14,0,4.0,34,101,0
2,3,Rebecca Glasscock,8,4,1.0,26,101,0
3,4,Shannel,5,6,-1.0,29,101,0
4,5,Ongina,14,0,4.0,26,101,0


In [74]:
dados_meio.to_excel("meio_posicao_queen.xlsx", index=False)

In [77]:
dados_fim= pd.merge(
    dados3,
    maxep_metadeep,
    left_on=["tempfranquia", "ep"],
    right_on=["tempfranquia", "maxep"],
    how="inner"
)
dados_fim=dados_fim.drop(['ep', 'metadeep', 'maxep','colocacao'], axis=1)
dados_fim.head()

,id,queen,bom,ruim,media,idade,tempfranquia,vencedora
0,1,BeBe Zahara Benet,18,7,3.0,28,101,1
1,2,Nina Flowers,22,4,5.0,34,101,0
2,3,Rebecca Glasscock,14,14,-1.0,26,101,0
3,4,Shannel,9,16,-4.0,29,101,0
4,10,Tyra Sanchez,35,5,9.0,21,201,1


In [78]:
dados_fim.to_excel("fim_posicao_queen.xlsx", index=False)